In [ ]:
# %load_ext autoreload
# %autoreload 2
# imports
import vame
import functions_bondjango as bd
import functions_vame as fv
import paths
import random
import os
import shutil
import h5py
import numpy as np
import importlib
import processing_parameters
import pandas as pd
import pprint as pp
from sklearn import cluster

In [ ]:
# test the batch function
importlib.reload(vame)
# importlib.reload(paths)

# define the test path
# test_path = [r'09_16_2021_11_03_39_miniscope_DG_210728_a_succ_head_petri']
test_path = [r'J:\Drago Guggiana Nilo\Prey_capture\AnalyzedData\09_08_2020_15_26_21_miniscope_DG_200701_a_succ_preproc.hdf5']

# load the data
# with pd.HDFStore(test_path) as h:
#     data_in = h['full_traces']
with h5py.File(test_path[0], 'r') as f:
    values = np.array(f['full_traces/block0_values'])
    labels = np.array(f['full_traces/block0_items']).astype(str)
    data = pd.DataFrame(values, columns=labels)

# get the list of columns
column_list_all = data.columns
column_list = [el for el in column_list_all if (('x' in el) or ('y' in el)) ]

# get the config path
config_file = os.path.join(paths.vame_path, paths.vame_current_model_name, 'config.yaml')

config = vame.read_config(config_file)

# test the alignment function
aligned_traj, frames = vame.egocentric_alignment(config, pose_ref_index=[0, 7], crop_size=(200, 200), 
                                                 use_video=False, video_format='.mp4', check_video=False, 
                                                 save_flag=False, filename=[data], column_list=column_list)
# aligned_traj = pd.DataFrame(aligned_traj.T, columns=column_list)
print(aligned_traj.shape)


In [ ]:
# Test the embedding
# import importlib
# importlib.reload(vame)
# pp.pprint(sys.modules)
# assemble the template path
first_file = os.listdir(os.path.join(paths.vame_results, 'results'))[0]
template_path = os.path.join(paths.vame_results, 'results', first_file, 'VAME', 'kmeans-15', 'cluster_center_'+first_file+'.npy')
# load the cluster centers
cluster_centers = np.load(template_path)
# get the number of clusters and features
n_cluster, _ = cluster_centers.shape
# create a new kmeans object
random_state = 42
n_init = 15
kmeans_object = cluster.KMeans(init='k-means++', n_clusters=n_cluster, random_state=random_state, n_init=n_init)
kmeans_object.cluster_centers_ = cluster_centers
kmeans_object._n_threads = 1
# run the pose segmentation
latents, clusters = vame.batch_pose_segmentation(config, [0], aligned_traj, kmeans_obj=kmeans_object)

print(latents[0].shape)
print(clusters[0].shape)